In [2]:
import pandas as pd
import csv

In [26]:
df = pd.read_pickle('logs/2023_06_2610_41_35/output2023_06_2610_41_35.pkl')

df["Visitadas"]


0                  https://pcyti.izt.uam.mx/?page_id=3831
1                   https://pcyti.izt.uam.mx/?page_id=175
2       https://pcyti.izt.uam.mx/wp-content/uploads/20...
3                   https://pcyti.izt.uam.mx/?page_id=153
4       https://pcyti.izt.uam.mx/wp-content/uploads/20...
                              ...                        
2602                                                     
2603                                                     
2604                                                     
2605                                                     
2606                                                     
Name: Visitadas, Length: 2607, dtype: object

In [18]:
s = df["Todos"].to_frame().T

In [21]:
print(df)

Empty DataFrame
Columns: [Todos, No_visitados, Visitados]
Index: []


In [28]:
with open("Documento", 'w', newline='') as myfile:
     wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
     wr.writerow(df)

In [22]:
import re
pattern = '^(?=.*http)(?=.*(?:expression|stack)).*$'
re.findall(pattern, 'https://stackoverflow.com/questions/3041320/regex-and-operator')




['https://stackoverflow.com/questions/3041320/regex-and-operator']

In [7]:
re.compile(pattern)

re.compile(r'(?=.*hi)(?=.*you)', re.UNICODE)

In [23]:
import asyncio
import httpx
from bs4 import BeautifulSoup

In [33]:
urls = ["http://books.toseefscrape.com/", 
        "http://books.toscrape.com/catalogue/page-2.html",
        "http://books.toscrape.com/catalogue/page-3.html",
        "http://books.toscrape.com/catalogue/page-4.html"]

async def fetch():
    async with httpx.AsyncClient() as client:
        tasks = (client.get(url) for url in urls)
        reqs = await asyncio.gather(*tasks)
    print("Finished")
    
    # soups = [BeautifulSoup(req, 'html.parser') for req in reqs]
    

In [34]:
asyncio.run(fetch())

RuntimeError: asyncio.run() cannot be called from a running event loop